In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from collections import deque
import random

2024-08-21 18:07:36.680080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class StockTradingEnv:
    def __init__(self, data):
        self.data = data
        self.reset()

    def reset(self):
        self.current_step = 0
        self.done = False
        self.state = self.data[self.current_step:self.current_step+sequence_length]
        return self.state

    def step(self, action):
        if self.done:
            return self.state, 0, self.done

        reward = 0
        self.current_step += 1

        if self.current_step >= len(self.data) - sequence_length:
            self.done = True

        next_state = self.data[self.current_step:self.current_step+sequence_length]
        
        # Reward calculation (simplified)
        if action == 1:  # Buy
            reward = self.data[self.current_step + sequence_length - 1] - self.data[self.current_step + sequence_length - 2]
        elif action == 2:  # Sell
            reward = self.data[self.current_step + sequence_length - 2] - self.data[self.current_step + sequence_length - 1]

        return next_state, reward, self.done

In [3]:
# Q-Network
def create_model(input_shape, action_space):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=action_space, activation='linear'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    return model

In [4]:
# Main training loop
def train(env, model):
    global epsilon
    for episode in range(num_episodes):
        state = env.reset()
        state = np.reshape(state, (1, sequence_length, 1))
        
        total_reward = 0
        while True:
            if np.random.rand() <= epsilon:
                action = random.randrange(3)  # Long, Short, Hold
            else:
                q_values = model.predict(state)
                action = np.argmax(q_values[0])

            next_state, reward, done = env.step(action)
            next_state = np.reshape(next_state, (1, sequence_length, 1))
            total_reward += reward

            memory.append((state, action, reward, next_state, done))
            state = next_state

            if done:
                print(f"Episode: {episode+1}/{num_episodes}, Reward: {total_reward}")
                break

            if len(memory) > batch_size:
                batch = random.sample(memory, batch_size)
                for state, action, reward, next_state, done in batch:
                    target = reward
                    if not done:
                        target += gamma * np.amax(model.predict(next_state)[0])
                    target_f = model.predict(state)
                    target_f[0][action] = target
                    model.fit(state, target_f, epochs=1, verbose=0)

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

## Hyperparameters

In [6]:
num_episodes = 1000
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_min = 0.1
epsilon_decay = 0.995
learning_rate = 0.001
batch_size = 32
memory_size = 2000
sequence_length = 10  # LSTM input sequence length

In [ ]:
# Experience Replay Memory
memory = deque(maxlen=memory_size)

In [7]:
data = pd.read_csv("../Binance Data/1h/BTCUSDT.csv")["Close"].values
env = StockTradingEnv(data)
model = create_model((sequence_length, 1), 3)
train(env, model)

/Users/taryn.tsui/Documents/Alvin/GitHub/alvinlui99/Algo Trading/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


NameError: name 'memory' is not defined